# LangChain 

LangChain is a framework for developing applications powered by large language models (LLMs).

## Installation

You could directly install the main library `langchain`and start experimenting with it. Or you could install all dependencies from a `requirement.txt` or `environments.yml` file.

Since I'm new in this AI/ML and GenAI world, I like to install package by package. I usually use `conda` and go from there to build my `environments.yml`. So in this case:  `conda install langchain -c conda-forge` 

You could also run your libraries one by one in your cell like this.

`%pip install langchain`.

## Architecture
Per Langchain documentation, the LangChain framework consists of multiple open-source libraries. Read more in the [Architecture](https://python.langchain.com/docs/concepts/architecture/)

We need to understand three of its main open-source libraries first to grass Langchain's concepts.
- `langchain-core`: base abstractions for chat models and other components. We will use here `messages` 
- integration language models: `langchain-openai`, `langchain-aws`, etc. 
- `langchain`: Chains, agents, and retrieval strategies

## LLM Application with ChatModels

LangChain supports many different language models that you can use interchangeably. Let's use the revolucionary `OpenAI`

`pip install -qU "langchain[openai]"`

`ChatModels`are instances of LangChain Runnables Interfaces. To simply call the model we pass a list of `messages` with the method `.invoke`

### Messages

`Messages` are objects used in propmts and chat conversations.

Check [Messages AI References](https://python.langchain.com/api_reference/core/messages.html)


To start a basic Chat, we can use the model directly by passing a list `SystemMessage` and `HummanMessage` messages.

Per definition `SystemMessage` is the message for priming AI behavior. How I see it, it's about defining the **persona** and **role** in some cases. In other cases, it's the **task** or instructions.

`HummanMessage` are messages that are passed in from a human to the model. How I see it "my Want" or **desire output**.



#### Initial steps

- Instal the **ChatModel** of your preferences. In my case: `%pip install -qU "langchain[openai]` 
- Set API_KEY, in this case  the `OPENAI_API_KEY`. For simplicity, I use `.env`
- Initialize the model


In [6]:
# %pip install langchain
# %pip install -qU "langchain[openai]

In [7]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")


In [8]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Spanish"),
    HumanMessage("Hi!, I'm Carmen. Welcome to my Langchain learning's journey"),
]

response = model.invoke(messages)

# It returns and AIMessage
response 

AIMessage(content='¡Hola! Soy Carmen. Bienvenido a mi viaje de aprendizaje de Langchain.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 31, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f7d56a8a2c', 'id': 'chatcmpl-BPcHa6jKQu5qGyUPydUPY2zE5AXhD', 'finish_reason': 'stop', 'logprobs': None}, id='run-901718c2-43d5-4472-b44b-c7b58bbd925c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 18, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
response.response_metadata

{'token_usage': {'completion_tokens': 18,
  'prompt_tokens': 31,
  'total_tokens': 49,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_f7d56a8a2c',
 'id': 'chatcmpl-BPcHa6jKQu5qGyUPydUPY2zE5AXhD',
 'finish_reason': 'stop',
 'logprobs': None}

In [10]:
for token in model.stream(messages):
    print(token.content, end="|")

|¡|Hola|!| Soy| Carmen|.| Bien|venido| a| mi| viaje| de| aprendizaje| sobre| Lang|chain|.||

## LLM Application using Prompt Templates

Coming from a FullStack Engineering background, I love this idea of templates! It brings good memories! 

Instead of passing a list of messages directly into the LM, we pass a template that it's a combination of user raw input and application logic. 

For that we will use `ChatPromptTemplates``


In [11]:
from langchain_core.prompts import ChatPromptTemplate

translation_systemp_template = "Translate the following from {from_language} into {to_language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", translation_systemp_template), ("user", "{text}")]
)

prompt = prompt_template.invoke({"from_language": "English", "to_language": "Spanish", "text": "We are learning the basics of LangChain"})

#It returns a ChatPromptValue
prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into Spanish', additional_kwargs={}, response_metadata={}), HumanMessage(content='We are learning the basics of LangChain', additional_kwargs={}, response_metadata={})])

In [12]:
# We can see now that we have a SystemMessage and HumanMessage like previously
prompt.to_messages()

[SystemMessage(content='Translate the following from English into Spanish', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='We are learning the basics of LangChain', additional_kwargs={}, response_metadata={})]

In [13]:
response = model.invoke(prompt)
print(response.content)

Estamos aprendiendo los conceptos básicos de LangChain.


#### Create a function 
So now we can put all the steps together in a function

In [15]:
def translate(from_language: str="English", to_language: str = "Spanish", text: str = "")-> str:
    translation_systemp_template = "Translate the following from {from_language} into {to_language}"

    prompt_template = ChatPromptTemplate.from_messages(
        [("system", translation_systemp_template), ("user", "{text}")]
    )

    if not text:
        text = "We are learning the basics of LangChain"

    # Generate the prompt
    prompt = prompt_template.invoke({
        "from_language": from_language, 
        "to_language": to_language, 
        "text": text
    })

    # Call the model
    response = model.invoke(prompt)
    return response.content

# Test the function
print(translate("English", "Spanish", "We are learning the basics of LangChain"))

Estamos aprendiendo los conceptos básicos de LangChain.


In [16]:
# Test the function with different languages
from_language = "English"
to_languages = ["Spanish", "French", "Italian", "Portuguese", "Guaraní"]

text = "Hello! Bienvenido! How are you?" 
print(" Text: ", text)
for language in to_languages:
    translated_text = translate(from_language, language, text)
    print(f"{from_language} -> {language}: {translated_text}")
print("="*50)

print(" Text: ", "We are learning the basics of LangChain")
for language in to_languages:
    translated_text = translate(from_language, language, "")
    print(f"{from_language} -> {language}: {translated_text}")

print("="*50)

text = "Thank you for stopping by in Internet's corner. I hope you have a great day!"
print(" Text: ", text)    
for language in to_languages:
    translated_text = translate(from_language, language, text)
    print(f"{from_language} -> {language}: {translated_text}")

 Text:  Hello! Bienvenido! How are you?
English -> Spanish: ¡Hola! ¡Bienvenido! ¿Cómo estás?
English -> French: Bonjour ! Bienvenue ! Comment ça va ?
English -> Italian: Ciao! Benvenuto! Come stai?
English -> Portuguese: Olá! Bem-vindo! Como você está?
English -> Guaraní: Hola! ¡Aña porã! Mba'éichapa?
 Text:  We are learning the basics of LangChain
English -> Spanish: Estamos aprendiendo los fundamentos de LangChain.
English -> French: Nous apprenons les bases de LangChain.
English -> Italian: Stiamo imparando le basi di LangChain.
English -> Portuguese: Estamos aprendendo o básico do LangChain.
English -> Guaraní: Ore nderehechávo LangChain arandukápe.
 Text:  Thank you for stopping by in Internet's corner. I hope you have a great day!
English -> Spanish: ¡Gracias por visitar el rincón de Internet! ¡Espero que tengas un gran día!
English -> French: Merci de vous être arrêté dans le coin d'Internet. J'espère que vous passez une excellente journée !
English -> Italian: Grazie per essere